In [98]:
import os
import re
import glob

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

LANGUAGES = ('en', 'it', 'fr', 'po', 'ru', 'ge')
LABELS = ('Economic', 'Capacity_and_resources', 'Morality', 'Fairness_and_equality',
          'Legality_Constitutionality_and_jurisprudence', 'Policy_prescription_and_evaluation', 'Crime_and_punishment',
          'Security_and_defense', 'Health_and_safety', 'Quality_of_life', 'Cultural_identity', 'Public_opinion',
          'Political', 'External_regulation_and_reputation')

# Load Data

In [79]:
base_dir = os.path.join('..', '..', '..','predictions')
models_dir = os.path.join(base_dir, 'best_models')
preds_dir = os.path.join(base_dir, 'predicted_instances')
preprocessed_dir = os.path.join('..', '..', '..', 'data', 'preprocessed')

In [80]:
os.listdir(preprocessed_dir)

['input_ru_train_and_dev.csv',
 'input_fr_train.csv',
 'input_ru_dev.csv',
 'input_fr_train_and_dev.csv',
 'input_ru_train.csv',
 'input_ge_train_and_dev.csv',
 'input_fr_dev.csv',
 'input_fr_test.csv',
 'input_ge_dev.csv',
 'input_it_train.csv',
 'input_en_train.csv',
 'input_it_test.csv',
 'input_it_train_and_dev.csv',
 'input_ge_test.csv',
 'input_po_test.csv',
 'input_it_dev.csv',
 'input_en_train_and_dev.csv',
 'input_en_test.csv',
 'input_po_train_and_dev.csv',
 'input_en_dev.csv',
 'input_ru_test.csv',
 'input_ge_train.csv',
 'input_po_dev.csv',
 'input_po_train.csv']

# Assess Performance of each model

# Voting Classifier

## en

In [93]:
# load ground truth
en_dev_df = pd.read_csv(os.path.join(preprocessed_dir, 'input_en_dev.csv'), index_col = 'id')
en_train_dev_df = pd.read_csv(os.path.join(preprocessed_dir, 'input_en_train_and_dev.csv'), index_col = 'id')
en_dev_df = en_dev_df[[]].join(en_train_dev_df)

In [96]:
mlb = MultiLabelBinarizer()
mlb.fit([LABELS])
y_dev = mlb.transform(en_dev_df.frames.str.split(','))
y_dev_df = pd.DataFrame(y_dev, columns=LABELS, index=en_dev_df.index)

In [120]:
train_dir = os.path.join(preds_dir, 'on_evalset_dev', 'en')
df = None
cols_to_use = list()
for i, train_df_path_i in enumerate(glob.glob(os.path.join(train_dir, "*.csv"))):
    exp_name = re.search(r'^[a-z]{2}_(([a-zA-Z]|_)+)+_y', os.path.basename(train_df_path_i)).group(1)
    preds_i_df = pd.read_csv(train_df_path_i, index_col='id').add_prefix(exp_name)
    f1_micro = f1_score(y_dev_df, preds_i_df, average='micro')

    if f1_micro > 0.67:
        print(exp_name)
        print(f"f1 micro: {f1_micro} \n")
        cols_to_use += list(preds_i_df.columns)

    if i == 0:
        df = preds_i_df
    else:
        df = df.join(preds_i_df)

SVM_sigmoid_BorderlineSMOTE
f1 micro: 0.6907730673316708 

ComplementNaiveBayes_ROS
f1 micro: 0.6804878048780487 



In [122]:
cols_to_use

['SVM_sigmoid_BorderlineSMOTEEconomic',
 'SVM_sigmoid_BorderlineSMOTECapacity_and_resources',
 'SVM_sigmoid_BorderlineSMOTEMorality',
 'SVM_sigmoid_BorderlineSMOTEFairness_and_equality',
 'SVM_sigmoid_BorderlineSMOTELegality_Constitutionality_and_jurisprudence',
 'SVM_sigmoid_BorderlineSMOTEPolicy_prescription_and_evaluation',
 'SVM_sigmoid_BorderlineSMOTECrime_and_punishment',
 'SVM_sigmoid_BorderlineSMOTESecurity_and_defense',
 'SVM_sigmoid_BorderlineSMOTEHealth_and_safety',
 'SVM_sigmoid_BorderlineSMOTEQuality_of_life',
 'SVM_sigmoid_BorderlineSMOTECultural_identity',
 'SVM_sigmoid_BorderlineSMOTEPublic_opinion',
 'SVM_sigmoid_BorderlineSMOTEPolitical',
 'SVM_sigmoid_BorderlineSMOTEExternal_regulation_and_reputation',
 'ComplementNaiveBayes_ROSEconomic',
 'ComplementNaiveBayes_ROSCapacity_and_resources',
 'ComplementNaiveBayes_ROSMorality',
 'ComplementNaiveBayes_ROSFairness_and_equality',
 'ComplementNaiveBayes_ROSLegality_Constitutionality_and_jurisprudence',
 'ComplementNaiveBaye

In [123]:
for i, label_i in enumerate(LABELS):
    if i == 0:
        voting_pred_df = pd.DataFrame(df[[col for col in df.columns if label_i in col and col in cols_to_use]].mean(axis=1).rename(label_i))
    else:
        voting_pred_df = voting_pred_df.join(df[[col for col in df.columns if label_i in col and col in cols_to_use]].mean(axis=1).rename(label_i))

In [124]:
# models with f1-score above 0.67
f1_score(y_dev_df, (voting_pred_df > 0.5).astype(int), average='micro')

0.6640106241699867

In [114]:
# models with f1-score above 0.65
f1_score(y_dev_df, (voting_pred_df > 0.5).astype(int), average='micro')

0.6621067031463749

In [109]:
# All models
f1_score(y_dev_df, (voting_pred_df > 0.5).astype(int), average='micro')

0.6471408647140865

In [59]:
voting_pred_df

,Economic
id,
832947852,0.0
814403783,0.0
830359136,0.0
813949697,0.0
833032366,0.0
...,...
832947600,0.0
832984694,0.0
832971448,0.0


# Sacked linear regression ensemble